# NB.
Сначала все ячейки до первой модели, потом начиная с обязательности полей и по "one by one", а потом возврат к первой модели.

In [54]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold

DISTRICT_NAMES_RU = ["Печерский", "Соломенский", "Деснянский", "Шевченковский", "Голосеевский", "Дарницкий", "Подольский", "Святошинский", "Оболонский", "Днепровский"] 

DISTRICT_NAMES_UA = ["Печерський", "Солом'янський", "Деснянський", "Шевченківський", "Голосіївський", "Дарницький", "Подільський", "Святошинський", "Оболонський", "Дніпровський"] 

METRO_RU = [u'Академгородок',u'Арсенальная',u'Берестейская',u'Бориспольская',u'Васильковская',u'Вокзальная',u'Выдубичи',u'Вырлица',u'Выставочный центр (ВДНХ)',u'Героев Днепра',u'Гидропарк',u'Голосеевская',u'Дарница',u'Дворец Украина',u'Дворец спорта',u'Демеевская',u'Днепр',u'Дорогожичи',u'Дружбы народов',u'Житомирская',u'Золотые Ворота',u'Ипподром',u'Кловская',u'Контрактовая площадь',u'Красный хутор',u'Крещатик',u'Левобережная',u'Лесная',u'Лукьяновская',u'Лыбидская',u'Майдан Незалежности',u'Минская',u'Нивки',u'Оболонь',u'Олимпийская (Республиканский стадион)',u'Осокорки',u'Петровка',u'Печерская',u'Площадь Льва Толстого',u'Позняки',u'Политехнический институт',u'Почтовая площадь',u'Святошино',u'Славутич',u'Сырец',u'Тараса Шевченко',u'Театральная',u'Теремки',u'Университет',u'Харьковская',u'Черниговская',u'Шулявская']

WALLS_RU = [u'Газоблок',u'Дерево/кирпич',u'Керамзито-бетон',u'Кирпич',u'Комбинир.',u'Монолит',u'Павильон (стекло)',u'Панель',u'Пенобетон',u'Пеноблок',u'Пеноблок/кирпич']
   
ROOMS_ARRANGEMENT = [u'\u0420', u'\u0421', u'\u0421-\u0440']

WC_TYPES = [ u'\u0420\u0430\u0437\u0434.', u'\u0421\u043e\u0432\u043c\u0435\u0441\u0442\u043d\u044b\u0439']

STATES = [u'\u0415\u0432\u0440\u043e\u0440\u0435\u043c\u043e\u043d\u0442', u'\u041a\u043e\u0441\u043c. \u0440\u0435\u043c\u043e\u043d\u0442', u'\u0420\u0435\u043c\u043e\u043d\u0442']

FLOOR_RU = [u'\u0414\u0435\u0440\u0435\u0432\u043e',  u'\u0414\u043e\u0441\u043a\u0430', u'\u041a\u043e\u0432\u0440\u043e\u043b\u0438\u043d',
 u'\u041a\u043e\u043c\u0431\u0438\u043d\u0438\u0440.',  u'\u041b\u0430\u043c\u0438\u043d\u0430\u0442',  u'\u041b\u0438\u043d\u043e\u043b\u0435\u0443\u043c',
 u'\u041f\u0430\u0440\u043a\u0435\u0442',  u'\u041f\u0430\u0440\u043a\u0435\u0442\u043d\u0430\u044f \u0434\u043e\u0441\u043a\u0430',
 u'\u041f\u043b\u0438\u0442\u043a\u0430', u'\u0421\u0442\u044f\u0436\u043a\u0430']

X_RU = [u'\u0414\u0435\u0440\u0435\u0432\u043e',
 u'\u0416\u0435\u043b\u0435\u0437\u043e\u0431\u0435\u0442\u043e\u043d',
 u'\u041a\u043e\u043c\u0431\u0438\u043d\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u044b\u0435']

DETAILS = [
    "square", 
           "kitchen_square",
#           "live_square", 
#           "rooms", 
           "floor_count", 
           "floor", 
#           'wc_count'
          ] #, 'levels']

NOMINAL_DETAILS = [
#    ('material', WALLS_RU),
#    ('rooms_arrangement',ROOMS_ARRANGEMENT),
#    ('state', STATES),
#    ('wc_type',WC_TYPES),
#    ('floor_material',FLOOR_RU),
#    ('x_material', X_RU),
]

YES_OR_NO_DETAILS = [
#    'parking', 'telephone', 'refrigerator', 'tvset', 'balcon'
]

FIELDS = DETAILS + sum([i[1] for i in NOMINAL_DETAILS],[]) + YES_OR_NO_DETAILS

FEATURES = DETAILS + DISTRICT_NAMES_RU + METRO_RU + WALLS_RU + ROOMS_ARRANGEMENT + STATES + WC_TYPES + FLOOR_RU + X_RU

USD = 24.5
EUR =  26.9

In [2]:
def district_vector(district_name):
    vec = [0]*len(DISTRICT_NAMES_RU)
    try:
        name = district_name.split()[0].encode('utf-8')
        vec[DISTRICT_NAMES_RU.index(name)] = 1
    except ValueError:
        try:
            vec[DISTRICT_NAMES_UA.index(name)] = 1
        except: 
            pass
    except:
        pass
    return vec

In [3]:
def metro_vector(metro_list):
    vec = [0]*len(METRO_RU)
    for metro_name in metro_list:
        try:
            vec[METRO_RU.index(metro_name)] = 1
        except:
            pass
    return vec

def nominal_vector(name, VALUES):
    vec = [0]*len(VALUES)
    try:
        vec[VALUES.index(name)] = 1
    except:
        pass
    return vec

def yes_or_no(ans):
    if ans.encode('utf-8') == 'да':
        return 1
    if ans.encode('utf-8') == 'Есть':
        return 1
    return 0

In [111]:
def extract_data(flat):
    X = []
    for d in DETAILS:
        try:
            X += [float(flat[d])]
            if d == 'live_square':
                if X[-1] > X[0]:
                    X[-1] = 0
            if d == 'kitchen_square':
                if X[-1] > X[0]:
                    X[-1] = 0
            if d == 'rooms':
                if X[-1] > 10:
                    X[-1] = 0
            if d == 'square':
                if X[-1] > 900:
                    X[-1] = 0
        except:
            X += [0]
    try:
        X += district_vector(flat['district'])
    except KeyError:
        X += [0] * len(DISTRICT_NAMES_RU)
    try:
        if type(flat['metro']) == list:
            X += metro_vector(flat['metro'])
        if type(flat['metro']) == dict:
            X += metro_vector(flat['metro'].keys())        
        X += [len(flat['metro'])]
    except KeyError:
        X += [0] * (len(METRO_RU)+1)
#        X += [0] 
    for n in NOMINAL_DETAILS:
        try:
            X += nominal_vector(flat[n[0]],n[1])
        except KeyError:
            X += [0] * len(n[1])
    for d in YES_OR_NO_DETAILS:
        try:
            X += [yes_or_no(flat[d])]
        except:
            X += [0]
    y=0
    try:
        y = float(flat['price'])
    except:
        print flat['code']
    try:
        if flat['currency'].lower().encode() == 'usd':
            y *= USD
        if flat['currency'].lower().encode() == 'eur':
            y *= EUR
    except:
        pass
    return [X, y]

In [5]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = []):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = 0.18, random_state = 3)
    regr = linear_model.LinearRegression(normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [6]:
#remove data without price
#field - str, data - list of dicts
def remove_with_no(field, t_data):
    c = 1
    while c!=0:
        c=0
        for d in t_data:
            try:
                d[field]
            except:
                #print d
                c+=1
                t_data.remove(d)
        print c
    return t_data

In [7]:
def get_X_y_from(t_data):
    t_cleaned_data = [extract_data(i) for i in t_data]
    X = np.array([i[0] for i in t_cleaned_data])
    y = np.array([i[1] for i in t_cleaned_data])
    return X,y

## Load data & clean data

In [84]:
with open('./spider_1000realty/realty_flats.json') as data_file: #last scray
    json_data = json.load(data_file)
print len(json_data)

realty_data = [i for i in json_data if i['city'].strip().encode('utf-8') in ["Киев","Київ"]]
print len(realty_data)

99282
99282


In [85]:
with open('metrovka_flats.json') as data_file:
#with open('metrovka_flats_metro.json') as data_file:
#with open('metrovka_flats_district.json') as data_file:    
    json_data = json.load(data_file)
print len(json_data)

#for d in json_data:
#    d['district']=[]

#filter city Kiyv
metrovka_data = [i for i in json_data ] #if i['city'].encode('utf-8') in ["Киев","Київ"]]
print len(metrovka_data)

10951
10951


In [86]:
print len(realty_data)
realty_data = remove_with_no('price', realty_data)
realty_data = remove_with_no('square', realty_data)
realty_data = remove_with_no('district', realty_data)
#realty_data = remove_with_no('metro', realty_data)
print len(realty_data)

99282
0
22
1
0
6013
920
81
8
2
0
92235


In [87]:
print len(metrovka_data)
metrovka_data = remove_with_no('price', metrovka_data)
metrovka_data = remove_with_no('square', metrovka_data)
metrovka_data = remove_with_no('district', metrovka_data)
#metrovka_data = remove_with_no('metro', metrovka_data)
print len(metrovka_data)

10951
0
0
2
0
10949


In [91]:
#merging
realty_data += metrovka_data
print len(realty_data)

114133


In [92]:
realty_data_1 = [i for i in realty_data if i['type'] == 1]
realty_data_2 = [i for i in realty_data if i['type'] == 2]

In [93]:
metrovka_data_1 = [i for i in metrovka_data if i['type'] == 1]
metrovka_data_2 = [i for i in metrovka_data if i['type'] == 2]

## First models

In [112]:
realty_X_1, realty_y_1 = get_X_y_from(realty_data_1)

In [113]:
realty_X_1[0]

array([ 56. ,   8.5,   0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   1. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         1. ,   0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   3. ])

In [114]:
regr = process_with(realty_X_1, realty_y_1, info=True)

Total: 4846, train: 3973, test: 873
Residual sum of squares: 172851644.76
Train absolute: 8131.87
Test absolute: 8136.84
Absolute to mean: 54.19%
Train variance score: 0.51
Test variance score: 0.51


In [110]:
metrovka_X_1, metrovka_y_1 = get_X_y_from(metrovka_data_1)

In [102]:
print("Residual sum of squares: %.2f"% np.mean((regr.predict(metrovka_X_1) - metrovka_y_1) ** 2))
print("Test absolute: %.2f"% np.mean(abs(regr.predict(metrovka_X_1) - metrovka_y_1)))
print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(metrovka_X_1) - metrovka_y_1))/np.mean(metrovka_y_1)*100))
print('Test variance score: %.2f' % regr.score(metrovka_X_1, metrovka_y_1))

Residual sum of squares: 183115860402.14
Test absolute: 28520.32
Absolute to mean: 83.47%
Test variance score: 0.01


In [103]:
realty_X_2, realty_y_2 = get_X_y_from(realty_data_2)

In [104]:
regr = process_with(realty_X_2, realty_y_2, info=True)

Total: 108861, train: 89266, test: 19595
Residual sum of squares: 523460945814595.44
Train absolute: 11025138.74
Test absolute: 8349668.03
Absolute to mean: 204.47%
Train variance score: 0.00
Test variance score: -0.18


In [105]:
metrovka_X_2, metrovka_y_2 = get_X_y_from(metrovka_data_2)

In [ ]:
print("Residual sum of squares: %.2f"% np.mean((regr.predict(metrovka_X_2) - metrovka_y_2) ** 2))
print("Test absolute: %.2f"% np.mean(abs(regr.predict(metrovka_X_2) - metrovka_y_2)))
print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(metrovka_X_2) - metrovka_y_2))/np.mean(metrovka_y_2)*100))
print('Test variance score: %.2f' % regr.score(metrovka_X_2, metrovka_y_2))

In [69]:
print "Features sorted by their score:"
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
#f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])

Features sorted by their score:
4380.32424708
98
489.906300 	 floor_count
172.325100 	 square
-182.477500 	 floor
-292.940900 	 kitchen_square


In [ ]:
#!-----extract frequency------
counts = [0] * len(FIELDS)
for d in realty_data:
    for k in d.keys():
        try:
            counts[FIELDS.index(k)]+=1
        except:
            pass

together = []
for i in range(0, len(FIELDS)):
    together += [[counts[i], FIELDS[i]]]

together.sort(reverse=True)

for i in range(0, len(FIELDS)):
    together[i] += [round(together[i][0]/float(len(realty_data))*100,2)]

for i in together:
    print i
#!---------------------------

## Plots

In [77]:
#show dots from all data
plt.scatter([np.linalg.norm(i) for i in realty_X_1], realty_y_1, color='black')
#plt.plot([np.linalg.norm(i) for i in X_2], regr.predict(X_2), color='blue', linewidth=1)
#plt.scatter(np.linalg.norm(X_2[ind]), y_2[ind], color='blue')
#plt.scatter([np.linalg.norm(i) for i in X_2[inx]], y_2[inx], color='blue')
plt.xticks(())
plt.yticks(())
plt.show()

##  Good vs bad

In [82]:
good = []
bad = []
all = zip(metrovka_X_2, metrovka_y_2)
for i,z in enumerate(all):
    r = regr.predict(z[0])
    err_procent = np.mean(abs(r - z[1])/z[1]*100)
    if err_procent > 30:
        bad += [[metrovka_data_2[i], r[0], err_procent]]
    else:
        good += [[metrovka_data_2[i], r[0], err_procent]]

NameError: name 'metrovka_X_2' is not defined

In [ ]:
print "Всего элементов: %d" % len(metrovka_data_2)
print "Плохих: %d \nХороших: %d" % (len(bad), len(good))
print "Процент плохих: %.2f \nПроцент хороших: %.2f" % (len(bad)/float(len(metrovka_data_2)), len(good)/float(len(metrovka_data_2)))
print "Среднее плохих: %.3f \nСреднее хороших: %.3f" % (np.mean(map(lambda x: x[2], bad)), np.mean(map(lambda x: x[2], good)))
print "Медиана плохих: %.3f \nМедиана хороших: %.3f" % (np.median(map(lambda x: x[2], bad)), np.median(map(lambda x: x[2], good)))

## Manual checking

In [ ]:
ind = 1129
#for i in metrovka_data_2[ind].values():
#    print i
#print metrovka_data_2[ind]
[x,y] = extract_data(metrovka_data_2[ind])
print metrovka_data_2[ind]['rooms']
print
print("%.2f \t diff"% np.mean((regr.predict(x) - y)))
print y, "real"
print regr.predict(x)[0]
print("%.2f"% np.mean(abs(regr.predict(x) - y)/y*100))

In [ ]:
[x,y] =extract_data({
 u'district': u'Оболонский',
 u'price': u'3000000.00',
 #u'rooms': u'2',
 u'square': u'80',
 u'type': 2,
})
print
print("%.2f \t diff"% np.mean((regr.predict(x) - y)))
print y, "real"
print regr.predict(x)[0]
print("%.2f"% np.mean(abs(regr.predict(x) - y)/y*100))

In [ ]:
for d in DISTRICT_NAMES_RU:
    [x,y] =extract_data({
     u'district': d.decode('utf-8'),
     u'price': u'3000000.00',
     #u'rooms': u'2',
     u'square': u'45',
     u'type': 2,
    })
    print "%s: \t %f" %(d,regr.predict(x)[0])

In [ ]:
for d in METRO_RU:
    [x,y] =extract_data({
     u'metro': [d],
     u'price': u'3000000.00',
     #u'rooms': u'2',
     u'square': u'45',
     u'type': 2,
    })
    print "%50s: \t %f" %(d,regr.predict(x)[0])

In [ ]:
from collections import Counter
data = Counter([i['square'] for i in metrovka_data])
data.most_common()   # Returns all unique items and their counts
data.most_common(2)

## Making some fields requiered

In [71]:
#remove_data_with_no_district_in_X
def make_field_required(t_data):
    ind = []
    X = get_X_y_from(t_data)[0]
    print "Elements: %d" % len(X)
    for i, x in enumerate(X):
        if np.count_nonzero(x[:])==0:
            ind += [i]
    print "Deleted: %d" % len(ind)
    ind = sorted(ind, reverse=True)
    for i in ind:
        t=t_data.pop(i)
    return t_data

In [72]:
realty_data_1=make_field_required(realty_data_1)

Elements: 2216
Deleted: 1


In [73]:
metrovka_data_1=make_field_required(metrovka_data_1)

Elements: 2098
Deleted: 15


## Delete data with big diffs

In [31]:
def mean_diff(t_data, normalize = True, plots = True, info=True):
    X, y = get_X_y_from(t_data)
    print "Before:"
    before = process_with(X, y, return_short=True)
    print before
    
    if normalize:
        min_max_scaler = preprocessing.MinMaxScaler()
        X = min_max_scaler.fit_transform(X)
        y = min_max_scaler.fit_transform(y)
    
    pts = np.float32(zip(map(lambda x: np.linalg.norm(x), X),y))
    
    mean_pt = np.mean(pts,axis=0)
    mean_x,mean_y = mean_pt
    diffs = np.apply_along_axis(np.linalg.norm,1,pts - mean_pt)
    mean_diff = np.mean(diffs)

    filtered = np.float32([pts[i] for i,diff in enumerate(diffs) if diff <= 2*mean_diff])
    filtered_mean = np.mean(filtered,axis=0)
    
    t_data_f = [t_data[i] for i,diff in enumerate(diffs) if diff <= 2*mean_diff]
    X_f, y_f = get_X_y_from(t_data_f)
    print "After:"
    after = process_with(X_f,y_f, return_short=True)
    print after
    
#    plt.plot(pts[:,0],pts[:,1],'ro')
#    plt.plot(mean_x,mean_y,'w^')
#    plt.plot(filtered[:,0],filtered[:,1],'bo')
#    plt.plot(filtered_mean[0],filtered_mean[1],'w^')
#    plt.show()
    if after[1] > before[1] or after[0] < before[0]:
        return t_data_f
    return t_data

## Delete "outlaws"

In [32]:
def get_ind_by_y(X, y, ind_max = True):
    if ind_max:
        return np.argmax(y)
    else:
        return np.argmin(y)

In [33]:
def get_ind_by_x_norm(X, y, ind_max=True):
    norms = [np.linalg.norm(i) for i in X]
    #sorted_norm = [np.linalg.norm(i) for i in X]
    #sorted_norm.sort()
    if ind_max:
        #ind = norms.index(sorted_norm[-1])
        ind = norms.index(max(norms))
    else:
        #ind = norms.index(sorted_norm[1])
        ind = norms.index(min(norms))
    return ind

In [34]:
def get_ind_by_y_to_x_norm(X, y, ind_max=True):
    y_to_norm_x = [y[i]/np.linalg.norm(X[i]) for i in range(0,len(X))]
    if ind_max:
        return np.argmax(y_to_norm_x)
    else:
        return np.argmin(y_to_norm_x)

In [35]:
def cut_data_by(t_data, get_ind_cut_function, ind_max_flag=True):
    previous_sum = -1000000
    previous_score = -100000
    X, y = get_X_y_from(t_data)
    [current_sum, current_score] = process_with(X, y, return_short=True)
    temp={}
    while current_score > previous_score or current_sum < previous_sum:
        previous_score = current_score
        previous_sum = current_sum

        inx = get_ind_cut_function(X, y,ind_max=ind_max_flag)
        #print t_data[norms.index(sorted_norm[-1])]['code'], sorted_norm[-1]
        temp = t_data.pop(inx)
        
        X, y = get_X_y_from(t_data)
        [current_sum, current_score] = process_with(X, y, return_short=True)
        
        print current_sum, previous_sum
        print current_score, previous_score
        print "-----"
    t_data+=[temp]
    return t_data

## One by one

In [ ]:
l = 1
while l!=len(realty_data_2):
    l = len(realty_data_2)
    print "by x norm max"
    realty_data_2 = cut_data_by(realty_data_2, get_ind_by_x_norm, True)
    print "by y max"
    realty_data_2 = cut_data_by(realty_data_2, get_ind_by_y, True)
    print "by y to x norm max"
    realty_data_2 = cut_data_by(realty_data_2, get_ind_by_y_to_x_norm, True)
    print "by x norm min"
    realty_data_2 = cut_data_by(realty_data_2, get_ind_by_x_norm, False)
    print "by y min"
    realty_data_2 = cut_data_by(realty_data_2, get_ind_by_y, False)
    print "by y to x norm min"
    realty_data_2 = cut_data_by(realty_data_2, get_ind_by_y_to_x_norm, False)
    print "mean_diff"
    realty_data_2 = mean_diff(realty_data_2, normalize = True)

In [97]:
l = 1
while l!=len(realty_data_1):
    l = len(realty_data_1)
    print "by x norm max"
    realty_data_1 = cut_data_by(realty_data_1, get_ind_by_x_norm, True)
    print "by y max"
    realty_data_1 = cut_data_by(realty_data_1, get_ind_by_y, True)
    print "by y to x norm max"
    realty_data_1 = cut_data_by(realty_data_1, get_ind_by_y_to_x_norm, True)
    print "by x norm min"
    realty_data_1 = cut_data_by(realty_data_1, get_ind_by_x_norm, False)
    print "by y min"
    realty_data_1 = cut_data_by(realty_data_1, get_ind_by_y, False)
    print "by y to x norm min"
    realty_data_1 = cut_data_by(realty_data_1, get_ind_by_y_to_x_norm, False)
    print "mean_diff"
    realty_data_1 = mean_diff(realty_data_1, normalize = True)

by x norm max
32581.8642756 38032.2257926
-1.05315357046 0.00807053864559
-----
36599.260417 32581.8642756
-0.0597957486042 -1.05315357046
-----
55415.4334876 36599.260417
0.011993376985 -0.0597957486042
-----
38872.5300613 55415.4334876
0.00370938013109 0.011993376985
-----
35898.8862894 38872.5300613
-0.373224567993 0.00370938013109
-----
36503.0181644 35898.8862894
-0.364453209683 -0.373224567993
-----
33833.6742248 36503.0181644
-1.14954125422 -0.364453209683
-----
35195.7339885 33833.6742248
-0.364804496653 -1.14954125422
-----
35558.0181059 35195.7339885
-0.380036557884 -0.364804496653
-----
by y max
30732.1706862 32960.3880932
-0.0293292665994 -0.653368342164
-----
22530.8758728 30732.1706862
-0.0323946507496 -0.0293292665994
-----
21743.8338525 22530.8758728
0.021259916691 -0.0323946507496
-----
17740.8286355 21743.8338525
0.0490116595362 0.021259916691
-----
17123.4019725 17740.8286355
0.131222944711 0.0490116595362
-----
18485.2899647 17123.4019725
0.0628574702614 0.131222944

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


21169.0531569 17973.0176854
0.0445761211624 0.0633403164346
-----
by y min
16762.6576743 17045.6136542
0.182624708564 0.083633962698
-----
18712.8485006 16762.6576743
0.0514334399919 0.182624708564
-----
by y to x norm min
18712.8485006 17682.9822689
0.0514334399919 0.070567279071
-----
mean_diff
Before:
(17682.982268884636, 0.070567279071024447)
After:


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


(16931.914452393106, 0.084322640541909522)
by x norm max
17633.2892955 16931.9144524
0.0565629020109 0.0843226405419
-----
by y max
18125.9639239 16931.9144524
0.0518374700823 0.0843226405419
-----
by y to x norm max
17483.8154691 15709.1450974
0.0891793757164 0.107876204724
-----
by x norm min
15180.8014867 18787.8432509
0.124093653972 0.0715501289838
-----
16308.810294 15180.8014867
0.0738284357229 0.124093653972
-----
by y min
16318.3637338 15883.5073023
0.0737468477424 0.0739716525215
-----
by y to x norm min
16318.3637338 15883.5073023
0.0737468477424 0.0739716525215
-----
mean_diff
Before:
(15883.507302291202, 0.073971652521454434)


...

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


After:
(8824.8301286579917, 0.47993557703363593)
by x norm max
8556.67115305 8824.83012866
0.467150145277 0.479935577034
-----
8465.1239519 8556.67115305
0.416780047426 0.467150145277
-----
8475.41829847 8465.1239519
0.472051854445 0.416780047426
-----
8950.05003989 8475.41829847
0.362057348225 0.472051854445
-----
by y max
9012.49422059 8445.17427222
0.429220547275 0.475260478389
-----
by y to x norm max
9015.18599532 9148.59397906
0.428993012293 0.398018571248
-----
9368.43467925 9015.18599532
0.347586714869 0.428993012293
-----
by x norm min
9405.6607852 9015.18599532
0.354661021108 0.428993012293
-----
by y min
9351.64746122 9015.18599532
0.344790056067 0.428993012293
-----
by y to x norm min
9351.64746122 9015.18599532
0.344790056067 0.428993012293
-----
mean_diff
Before:
(9015.1859953241292, 0.42899301229295289)


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


After:
(8271.2699807220961, 0.42669475449190286)
by x norm max
8478.57581757 8271.26998072
0.423003724193 0.426694754492
-----
by y max
8751.33346529 8271.26998072
0.387899245259 0.426694754492
-----
by y to x norm max
8753.58121058 8571.79761815
0.387611441551 0.465142219618
-----
by x norm min
8748.9677069 8571.79761815
0.387616608464 0.465142219618
-----
by y min
8755.49856076 8571.79761815
0.387608610838 0.465142219618
-----
by y to x norm min
8755.49856076 8571.79761815
0.387608610838 0.465142219618
-----
mean_diff
Before:
(8571.7976181484555, 0.46514221961815244)
After:
(9384.1327833275118, 0.4296563930981363)


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


## Saving and loading coefs

In [ ]:
coefs = map(lambda x: round(x, 4), regr.coef_)
f = zip(coefs, FEATURES)
print coefs

In [ ]:
import pickle
with open('filtered_mixed_coefs_district_square_required.json', 'w') as outfile:
    pickle.dump(coefs, outfile)

In [ ]:
import pickle
with open('filtered_mixed_coefs_district_square_required.json', 'r') as infile:
    c = pickle.load(infile)

## Save data

In [ ]:
with open('metrovka_flats_metro.json', 'w') as outfile:
    json.dump(metrovka_data, outfile)
#json.dumps(data_2[0:1])